<a href="https://colab.research.google.com/github/Tejas-Panchal/ai_greenskill_edunet_foundation/blob/main/2025_07_10_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>